# Generic Pipeline tests

## Imspector-independent stuff

In [ ]:
import numpy as np
from itertools import count
from pprint import pprint
import json
from queue import PriorityQueue
from collections import defaultdict
from time import time, sleep, clock
import signal
import hashlib
import os
from itertools import zip_longest, chain
from unittest.mock import MagicMock


In [ ]:
class AcquisitionPriorityQueue(PriorityQueue):
    """
    slightly modified PriorityQueue to be able to enqueue non-orderable data
    """
    def __init__(self):
        PriorityQueue.__init__(self)
        self.ctr = count()
    def put(self, item, prio):
        PriorityQueue.put(self, (prio, next(self.ctr), item))
    def get(self, *args, **kwargs):
        lvl, _, item = PriorityQueue.get(self, *args, **kwargs)
        return (lvl, item)
        
class _pipeline_level:
    """
    named level in an acquisition pipeline
    should not be used outside of a PipelineLevels object
    """
    def __init__(self, parent, name):
        self.parent = parent
        self.name = name
    
    def __eq__(self, other):
        if type(self) != type(other):
            return False
        return self.name == other.name
    
    def __le__(self, other):
        return self.parent.reversedLevels.index(self) <= self.parent.reversedLevels.index(other)
    
    def __lt__(self, other):
        return self.parent.reversedLevels.index(self) < self.parent.reversedLevels.index(other)
    
    def __str__(self):
        return self.name
    
    def __hash__(self):
        return str.__hash__(self.name)
    
    def __repr__(self):
        return self.name


class DelayedKeyboardInterrupt():
    """
    context manager to allow finishing of one acquisition loop
    before quitting queue due to KeyboardInterrupt
    
    modified from https://stackoverflow.com/a/21919644
    """
    
    def __init__(self, pipeline):
        self.pipeline = pipeline
    
    def __enter__(self):
        self.old_handler = signal.getsignal(signal.SIGINT)
        signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        self.pipeline.interrupted = True

    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
    
class PipelineLevels:
    """
    ordered collection of _pipeline_level
    """
    levels = []
    def __init__(self, *args):
        for arg in args:
            lvl = _pipeline_level(self, arg)
            self.levels.append(lvl)
            setattr(self, arg, lvl)
    @property
    def reversedLevels(self):
        return list(reversed(self.levels))

class TimedStoppingCriterion():
    """
    stopping criterion to stop after a set amount of time
    """
    def __init__(self, maxtime):
        self.maxtime = maxtime
    def check(self, pipeline):
        return time() > ( pipeline.startingTime + self.maxtime )
    def desc(self, pipeline):
        return 'STOPPING PIPELINE {}: maximum time exceeded'.format(pipeline.name)
        
class InterruptedStoppingCriterion():
    """
    stopping criterion to check wether SIGINT was received and stop then
    will also reset the signal status in parent AcquisitionPipeline
    """
    def check(self, pipeline):
        return pipeline.interrupted
    def resetInterrupt(self, pipeline):
        pipeline.interrupted = False
    def desc(self, pipeline):
        return 'STOPPING PIPELINE {}: interrupted by user'.format(pipeline.name)

class AcquisitionTask():
    """
    a dummy acquisition task, that will repeat itself every second
    """
    
    def __init__(self, pipelineLevel):
        self.pipelineLevel = pipelineLevel
        self.measurementUpdates = []
        self.settingsUpdates = []
        self.nAcquisitions = 0
        self.delay = 0
        
    def withUpdates(self, updates):
        measUpdates = [m for m,s in updates]
        settingsUpdates = [s for m,s in updates]
        self.measurementUpdates.append(measUpdates)
        self.settingsUpdates.append(settingsUpdates)
        return self
    
    def withDelay(self, delay=0):
        self.delay = delay
        return self
        
    @property
    def numAcquisitions(self):
        return self.nAcquisitions
    
    def getUpdates(n):
        return self.measurementUpdates[n], self.settingsUpdates[n]
    
    def getAllUpdates():
        return [self.getUpdates(n) for n in range(self.numAcquisitions)]
    
    # TODO: move, this should become part of an analysis callback
    def __call__(self, pipeline, *args, **kwargs):
        print('pipeline {}: do dummy acquisition on level {}'.format(pipeline.name, self.pipelineLevel))
        #sleep(1)
        pipeline.queue.put(AcquisitionTask(self.pipelineLevel).withDelay(self.delay), self.pipelineLevel)

# TODO: implement
class RichData:
    def __init__(self):
        self.globalSettings = []
        self.measurementSettings = []
        self.data = []
    
    # TODO: remove defaults
    def append(self, globalSettings=None, measurementSettings=None, data=None):
        self.globalSettings.append(globalSettings)
        self.measurementSettings.append(measurementSettings)
        self.data.append(data)
    
    @property
    def numConfigurations(self):
        return len(self.data)
    
    def numImages(self, n):
        if n < self.numConfigurations:
            return len(self.data[n])
        else:
            return 0
    

class DefaultNameHandler():
    
    def __init__(self, path, levels, prefix=None):
        self.path = path
        self.levels = levels
        if prefix is None:
            hash_object = hashlib.md5(bytes(str(time()), "utf-8"))
            hex_dig = hash_object.hexdigest()
            self.prefix = str(hex_dig)
        else:
            self.prefix = prefix
            
    def _mkdir_if_necessary(self):
        pass
            
    def get_filename(self, idxes):
        insert = chain.from_iterable(zip([l.name for l in self.levels.levels[0:len(idxes)]], idxes))
        insert = list(insert)
        return ((self.prefix + '_{}_{}' * len(idxes)).format(*insert))
    
    def get_path(self, idxes):
        return os.path.join(self.path, self.get_filename(idxes))      
        
            
class MockImspectorConnection():
    
    def __init__(self):
        self.getCurrentData = MagicMock(return_value = RichData())
        self.makeMeasurementFromTask = MagicMock(return_value = None)
        self.makeConfigurationFromTask = MagicMock(return_value = None)
        self.runCurrentMeasurement = MagicMock(return_value = None)
        self.saveCurrentMeasurement = MagicMock(return_value = None)
        self.closeCurrentMeasurement = MagicMock(return_value = None)
            

class AcquisitionPipeline():
    
    
    
    def __init__(self, name):
        """
        construct with name
        """
        self.name = name
        
        self.pipelineLevels = None
    
        # we habe an InterruptedStoppingCriterion by default
        self.stoppingConditions = [InterruptedStoppingCriterion()]
        self.queue = None
        self.startingTime = None
        self.counters = defaultdict(int)
        self.data = defaultdict(RichData)
        self.callbacks = defaultdict(list)
        
        # hold the Imspector connection
        self.im = MockImspectorConnection()
        
        self.logger = None
        self.nameHandler = None
    
        # the DelayedKeyboardInterrupt will indicate a received SIGINT here
        self.interrupted = False
    
    def run(self):
        """
        run the pipeline
        """
        
        # we use this context manager to handle interrupts so we can finish
        # to acquisition we are in before stopping
        with DelayedKeyboardInterrupt(self):            

            # record starting time, so we can check wether a StoppingCondition is met
            self.startingTime = time()

            lvl = None
            
            while not self.queue.empty():

                # get next task and its level
                oldlvl = lvl
                lvl, acquisition_task = self.queue.get()
                
                # TODO: remove Mock
                acquisition_task.getUpdates = MagicMock(return_value=(None, None))
                acquisition_task.nAcquisitions = MagicMock(return_value = 1)
                
                # reset or increment indices
                if (oldlvl != lvl):
                    self.counters[lvl] = 0
                else:
                    self.counters[lvl] += 1
                                
                # create index of measurement (indices of all levels until lvl)
                currentMeasurementIdx = tuple((self.counters[l] for l in self.pipelineLevels.levels[
                            0:self.pipelineLevels.levels.index(lvl)+1]))

                # go through updates sequentially (we might have multiple configurations per measurement)
                for updatesI in range(acquisition_task.numAcquisitions):
                    
                    # update imspector
                    if updatesI == 0:
                        self.im.makeMeasurementFromTask(acquisition_task.getUpdates(updatesI))
                    else:
                        self.im.makeConfigurationFromTask(acquisition_task.getUpdates(updatesI))
                    
                    # we might want to sleep
                    sleep(acquisition_task.delay)
                    
                    # run in imspector
                    self.im.runCurrentMeasurement()
                                
                    # add data copy (of most recent configuration) to internal storage
                    self.data[currentMeasurementIdx].append(self.im.getCurrentData())
                
                # save and close in imspector
                path = None
                if self.nameHandler != None:
                    path = self.nameHandler.get_path(currentMeasurementIdx)
                
                # TODO: closing without saving might trigger UI dialog in Imspector
                if not (path is None):
                    self.im.saveCurrentMeasurement(path)
                self.im.closeCurrentMeasurement()

                # do the callbacks (this should do analysis and re-fill the queue)
                callbacks_ = self.callbacks.get(lvl, None)
                if not (callbacks_ is None):
                    for callback_ in callbacks_:                    
                        callback_(self)

                # go through stopping conditions
                for sc in self.stoppingConditions:
                    if sc.check(self) == True:
                        # reset interrupt flag if necessary
                        if isinstance(sc, InterruptedStoppingCriterion):
                            sc.resetInterrupt(self)
                        print(sc.desc(self))
                        break
                # we went through all the loop iterations (no break)
                else:
                    continue
                break
            
            print('PIPELINE {} FINISHED'.format(self.name))


              
    def withPipelineLevels(self, lvls):
        """
        set pipeline levels, can be chained
        """
        self.pipelineLevels = lvls
        return self
    
    def withNameHandler(self, nh):
        self.nameHandler = nh
        return self
    
    def withImspectorConnection(self, im):
        self.im = im
        return self
    
    def withCallbackAtLevel(self, callback, lvl):
        """
        set the callback for a level, can be chained
        """
        if not (lvl in self.pipelineLevels.levels):
            raise ValueError('{} is not a registered pipeline level'.format(lvl))
        self.callbacks[lvl].append(callback)
        return self
    
    def _withStoppingConditions(self, conds):
        """
        reset the StoppingConditions, can be chained
        """
        self.stoppingConditions.clear()
        for condI in conds:
            self.stoppingConditions.append(condI)
        return self
    
    def withAddedStoppingCondition(self, cond):
        """
        add a StoppingCondition, can be chained
        """
        self.stoppingConditions.append(cond)
        return self
    
    def withInitialTask(self, task, lvl):
        """
        initialize the queue with the given task at the given level, can be chained
        """
        self.queue = AcquisitionPriorityQueue()
        self.queue.put(task, lvl)
        return self
    
                
                
                
            
            
pll = PipelineLevels('overview', 'detail', 'sted')
tsk = AcquisitionTask(pll.overview).withDelay(.0)

pl = (AcquisitionPipeline('1')
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('aaa', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(5))
        .withInitialTask(tsk, pll.overview)
        .withCallbackAtLevel(AcquisitionTask(pll.overview).withDelay(.3), pll.overview))
pl.run()


'''
pl = (AcquisitionPipeline('2')
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('aaa', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(5))
        .withInitialTask(tsk, pll.overview))
pl.run()
'''  

pl.im.getCurrentData.mock_calls
pl.data

# Imspector stuff from here on

In [ ]:
import specpy as sp
im = sp.Imspector()
im.version()

In [ ]:
it = count()


ms = im.create_measurement()
ms.set_parameters('', params)

im.connect_begin(lambda : next(it), 1)
im.run(ms)

params = ms.parameters('')
js = json.dumps(ms.parameters(''), indent=2)

im.close(ms)
print(next(it))

print(js)
justoix = dict([(k,v) for k,v in params.items() if (k == 'OlympusIX')])

#pprint(im.parameters(''))